# From DIY to TensorFlow 1.x to TensorFlow 2.x 
此處三個範例均為採用鳶尾花資料集的預測問題
1. All DIY (Hard Coding)
2. TensorFlow 1.x
3. TensorFlow 2.x (Keras API)

## 1. All DIY (Hard Coding)

In [ ]:
import numpy as np
import pprint
from sklearn import datasets
from scipy import special
pp = pprint.PrettyPrinter(indent=4)

def softmax(x,der=False):
  if der:
    return np.exp(x)*(np.sum(np.exp(x))-np.exp(x)) / np.power(np.sum(np.exp(x)),2)
  e = np.exp(x - np.max(x, axis=1).reshape((-1, 1)))
  return e / e.sum(axis=1).reshape((-1, 1))

def sigmoid(x,der=False):
  if der:
    return x*(1-x)
  else:
    return special.expit(x)

def generate_batch(data,batch_size):
  data_set = []
  batch = []
  for i in range(0,len(data)):
    if i!=0 and i%batch_size==0:
      data_set.append(np.asarray(batch))
      batch = []
    batch.append(data[i])
  return data_set

def random_sample(input, output):
  validation_input = []
  validation_output = []
  for i in range(0,10):
    picked_number=np.random.randint(0,len(input))
    validation_input.append(input[picked_number])
    validation_output.append(output[picked_number])
    input = np.delete(input, picked_number, 0)
    output = np.delete(output, picked_number, 0)
  return validation_input,validation_output

def int_to_vector(data,label_size):
  vector = np.zeros((len(data),label_size),dtype='f')
  for i,single in enumerate(data):
    vector[i][single]=1
  return vector

feature_size = 4  # input layer
label_size = 3  # output layer
second_layer_size = 4  # hidden layer

iris = datasets.load_iris()
input = iris['data'][:,:feature_size]
output = iris['target']

np.random.seed(1)

w1 = 2 * np.random.random((feature_size,second_layer_size))-1 #4,4
w2 = 2 * np.random.random((second_layer_size,label_size))-1 #4,3
epoch = 30000

new_output=[]
for value in output:
  new_output.append([value])
output = np.asarray(new_output)

validation_input,validation_output=random_sample(input,output)
for ep in range(0,epoch):
  label_vector = int_to_vector(output,label_size)
  l1 = input
  l2 = sigmoid(np.dot(l1,w1))
  l3 = softmax(np.dot(l2,w2))
  l3_error = label_vector-l3
  l3_delta = l3_error*softmax(l3,True)
  l2_error = l3_delta.dot(w2.T)
  l2_delta = l2_error*sigmoid(l2,True)
  w2 += l2.T.dot(l3_delta)
  w1 += l1.T.dot(l2_delta)
  if ep%500==0:
    print(np.mean(np.abs(l3_error)))

pp.pprint([np.argmax(x) for x in l3])
pp.pprint(label_vector)

def predict(validation_input,validation_output):
  l1 = validation_input
  l2 = sigmoid(np.dot(l1, w1))
  l3 = softmax(np.dot(l2, w2))
  print([np.argmax(x) for x in l3])
  print([x[0] for x in np.asarray(validation_output)])

predict(validation_input,validation_output)


## 2. TensorFlow 1.x

In [ ]:
# 強制指定使用 1.x 版
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# import tensorflow as tf
from sklearn import datasets
import numpy as np

def int_to_vector(data,label_size):
  vector = np.zeros((len(data),label_size),dtype='f')
  for i,single in enumerate(data):
    vector[i][single]=1
  return vector

iris = datasets.load_iris()
input = iris['data']
output = iris['target']
feature_size = 4
second_layer_size = 4
label_size = 3
output=int_to_vector(output,label_size)
tf.set_random_seed(1)
train_inputs = tf.placeholder(tf.float32, shape=[None,feature_size])
train_labels = tf.placeholder(tf.float32, shape=[None,label_size])
w1 = tf.Variable(tf.random_uniform([feature_size, second_layer_size], -1.0, 1.0))
w2 = tf.Variable(tf.random_uniform([second_layer_size, label_size], -1.0, 1.0))
pass_1 = tf.nn.sigmoid(tf.matmul(train_inputs,w1))
vali = tf.matmul(pass_1,w2)
pass_2 = tf.nn.softmax(tf.matmul(pass_1,w2))

loss = tf.reduce_mean(tf.abs(train_labels-pass_2))
session = tf.Session()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
init = tf.global_variables_initializer()
session.run(init)

epoch = 30000
feed_dict = {train_inputs: input, train_labels: output}
for i in range(0,epoch):
    pred, mid, intt, _, cur_loss = session.run([pass_2,pass_1,train_inputs,optimizer, loss], feed_dict=feed_dict)
    if i%1000==0:
        print(cur_loss)

print([np.argmax(x) for x in pred])


## 3. TensorFlow 2.x (Keras API)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from sklearn import datasets

# 載入資料集
iris = datasets.load_iris()
x = iris.data  # iris['data']
y = iris.target  # iris['target']

# 分割成訓練和測試資料集 (直接切割是錯誤的!!!)
# x_train, y_train = x[:120], y[:120]     # 訓練資料前120筆
# x_test, y_test = x[120:], y[120:]      # 測試資料後30筆

# 打亂資料集中的所有資料 (permutation函式: 隨機排列)
i = np.random.permutation(len(iris.data))

# 分割成訓練和測試資料集 (依隨機順序)
x_train = x[i[:120]]   # 前120條資料
y_train = y[i[:120]]   # 前120條資料對應的花的型別
x_test = x[i[120:]]   # 最後30條資料
y_test = y[i[120:]]   # 最後30條資料對應的花的型別

# 對輸出進行 onehot 編碼
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

# 建立Keras的Sequential模型
model = Sequential()
model.add(Dense(6, input_shape=(4,), activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='softmax'))
# model.summary()   # 顯示模型摘要資訊

# 編譯模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
model.fit(x_train, y_train_onehot, epochs=100, batch_size=5)

# 評估模型
loss, accuracy = model.evaluate(x_test, y_test_onehot, verbose=0)
print('準確度 = {:.2f}'.format(accuracy))


## 練習